In [0]:
%pip install requests beautifulsoup4

In [0]:
from pyspark.sql import SparkSession
import requests

# Initialize Spark session
spark = SparkSession.builder \
    .appName("RemoteOK Jobs") \
    .getOrCreate()

# Get data from API
url = "https://remoteok.io/api"
response = requests.get(url)
jobs = response.json()

# First element is metadata, skip it
jobs_data = jobs[1:]

# Create PySpark DataFrame from the list of dicts
jobs_df = spark.createDataFrame(jobs_data)

# Show the schema and first few rows
jobs_df.printSchema()
display(jobs_df)

# Count total rows
print("Total rows:", jobs_df.count())


In [0]:
# Set the default catalog and schema for this Spark session
spark.sql("USE CATALOG etl_catalog")
spark.sql("USE SCHEMA etl_db")


In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS etl_db")
jobs_df.write.format("delta").mode("overwrite").saveAsTable("etl_catalog.etl_db.remoteok_bronze")

In [0]:
%sql
describe etl_catalog.etl_db.remoteok_bronze